In [37]:
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor,as_completed, wait# For Multiprocessing


In [13]:
df_scanner = pd.read_csv(r"./scanner_data/scanner_data.csv")


In [14]:
df_scanner.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [15]:
df_scanner.rename(columns={"Unnamed: 0":"Row ID"},inplace=True)

In [16]:
df_scanner.columns

Index(['Row ID', 'Date', 'Customer_ID', 'Transaction_ID', 'SKU_Category',
       'SKU', 'Quantity', 'Sales_Amount'],
      dtype='object')

In [17]:
# Check for duplicates
df_scanner.duplicated().sum()

0

In [18]:
# Check for missing values
df_scanner.isnull().sum()

Row ID            0
Date              0
Customer_ID       0
Transaction_ID    0
SKU_Category      0
SKU               0
Quantity          0
Sales_Amount      0
dtype: int64

In [19]:
# Check datatypes
df_scanner.dtypes

Row ID              int64
Date               object
Customer_ID         int64
Transaction_ID      int64
SKU_Category       object
SKU                object
Quantity          float64
Sales_Amount      float64
dtype: object

The total revenue generated by the store

In [20]:
# Function to calculate revenue for a single sale
def calculate_revenue(sale):
    return sale['Quantity'] * sale['Sales_Amount']

In [21]:
# Function to calculate revesales_datanue using multiprocessing:
def calculate_total_revenue(sales_data):
    # Create a threadpool executor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit a calculation job for each sale
        results = [executor.submit(calculate_revenue,sale) for i,sale in sales_data.iterrows()]

        # Collect the result 
        revenue_per_sale = [i.result() for i in concurrent.futures.as_completed(results)]

    # calculate the total revenue
    total_revenue = sum(revenue_per_sale)

    return total_revenue

In [22]:
# Calling the function to calculate total revenue
total_revenue = calculate_total_revenue(df_scanner)

In [23]:
print("Total revenue : ", total_revenue)

Total revenue :  2760509.3001500685


The top 10 customers who have spent the most money at the store.

In [24]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Calculate spending of a single customer
def calculate_spend(cust_sale):
    return cust_sale['Quantity'] * cust_sale['Sales_Amount']

In [25]:
# Calculate top 10 customer spents concurrently
def calculate_top_customers(sales_data):
    customer_spending = sales_data.groupby(by='Customer_ID').agg({"Quantity" : "sum",
                                                                  "Sales_Amount":"sum"}).reset_index()
    # customer_spending["spending"] = calculate_spend(customer_spending)


    # Create a threadpool executor
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(calculate_spend,cust_sales) for i,cust_sales in customer_spending.iterrows()]
        customer_spending['spending'] = [future.result() for future in as_completed(futures)]

     # Sort the customers and return top 10   
    top10 = customer_spending.sort_values(by="spending",ascending=False).head(10)

    return top10

In [26]:
# Call function
top_customers = calculate_top_customers(df_scanner)

In [27]:
# Print the top 10 customers
print('Top 10 customers ID  by spending:')
print(top_customers)

Top 10 customers ID  by spending:
       Customer_ID  Quantity  Sales_Amount     spending
21414        21415       6.0         22.23  3248142.506
9071          9072       4.0         14.56  1566825.275
17043        17044       1.0          3.85   936682.974
15628        15629       5.0         13.37   777906.250
10878        10879      14.0         81.52   641999.280
5700          5701       5.0         20.33   533245.680
3078          3079       4.0         58.74   508010.400
18826        18827      17.0         82.45   490144.900
1359          1360       8.0          9.68   485796.620
12914        12915      11.0         62.90   480588.965


The total number of transactions that have taken place

In [28]:
# Function to count the total number of transactions
def count_transaction(df):

    return df['Transaction_ID'].nunique()
    # return len(list(set(df['Transaction_ID'].values.tolist())))

In [29]:
# Split the dataframe into chunks
chunk_size = len(df_scanner)//4 # Splitting into 4 chunks
chunks = [df_scanner[i:i+chunk_size] for i in range(0,len(df_scanner)+1,chunk_size)]


In [30]:
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(count_transaction,chunk) for chunk in chunks]

    # Wait for all futures to complete
    concurrent.futures.wait(futures)

    # Get results and sum them up
    total_transactions = sum([future.result() for future in futures])

print("Total number of transactions:", total_transactions)

Total number of transactions: 64685


The top 10 items that have been sold the most in terms of quantity

In [65]:
from collections import defaultdict
sku = df_scanner['SKU'].values.tolist()
quantity = df_scanner['Quantity'].values.tolist()
# create dictionary with concurrent.futures
result_dict = defaultdict(list)
with ThreadPoolExecutor() as executor:
    futures = []
    for k, v in zip(sku, quantity):
        futures.append(executor.submit(lambda x: result_dict[x[0]].append(x[1]), (k, v)))
    wait(futures)

# Summing up the values in the list of values
result_dict = {k:sum(v) for k, v in result_dict.items()}
# Sorting in descending order
result_dict = dict(sorted(result_dict.items(), key=lambda x: x[1], reverse=True))
# print result
print(list(result_dict.items())[0:10])

[('CKDW0', 5769.2), ('TD3DD', 3786.0), ('UNJKW', 2179.0), ('EEI1Q', 1568.0), ('MXKDP', 1252.0), ('H8P2L', 1118.0), ('2SVKS', 1113.0), ('CYRX4', 1071.0), ('W1ZMG', 1033.0), ('C6TXL', 1027.0)]
